Try to find region of interest in undistorted image

In [ ]:
import os
import cv2
import pickle
import matplotlib.image as mpimg

project_dir = os.path.dirname(os.getcwd())
test_folder_path = os.path.join(project_dir, 'test_images')
output_folder_path = os.path.join(project_dir, 'output_images')

pickle_path = os.path.join(project_dir, 'dist_pickle.p')
dist_pickle = pickle.load(open(pickle_path, "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

In [ ]:
import sys
sys.path.append('../')
from src.Frame import Frame

Frame.init(1280, 720, mtx, dist)

Create warp image to test

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline

image_pattern = os.path.join(test_folder_path, '*.jpg')
images = glob.glob(image_pattern)

def get_bird_view_image(image_name):
    image_path = os.path.join(test_folder_path, image_name)
    img = mpimg.imread(image_path)
    
    frame = Frame(img)
    
    thresholded_image = frame._preprocess_image()
    warped_image = frame._bird_view_transformation(thresholded_image)
    
    output_image_path = os.path.join(output_folder_path, 'bv_'+image_name)
    mpimg.imsave(output_image_path, warped_image, cmap="gray")
    
    plt.figure();
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(36, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(thresholded_image, cmap='gray')
    ax2.set_title('Threshold Image', fontsize=50)
    ax3.imshow(warped_image, cmap='gray')
    ax3.set_title('Bird-view Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
for name in images:
    get_bird_view_image(os.path.basename(name))

Find lines using convolution approach

In [ ]:
import numpy as np
from src.ConvolutionalSlider import ConvolutionalSlider


# window settings
window_width = 50 
window_height = 240 # Break image into 9 vertical layers since image height is 720
margin = 170 # How much to slide left and right for searching
conv_threshold = 50

def window_mask(width, height, img_ref, center, y):
    output = np.zeros_like(img_ref)
    output[int(y-0.5*height):int(y+0.5*height)
           ,max(0, int(center-width/2)):min(int(center+width/2), img_ref.shape[1])] = 1
    return output

def find_line_centroids(warped, l_center, y, conv_value, window_width, window_height, margin):
    window_centroids = [l_center] # Store the (left,right) window centroid positions per level
    y_values = [y]
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    conv = [conv_value]
    direction = 0
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),
                                    :], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-(margin if direction<=0 else int(margin/2)), 0))
        l_max_index = int(min(l_center+offset+(margin if direction>=0 else int(margin/2)), warped.shape[1]))
        l_conv=conv_signal[l_min_index:l_max_index]
        #reached the end of the line, 
        
        new_center = (np.argmax(l_conv)+l_min_index-offset) if np.max(l_conv) > conv_threshold else l_center
        angle = abs((level*window_height)/(new_center-window_centroids[0])) if (new_center-window_centroids[0]) != 0 else 11
        #print(np.max(l_conv), l_min_index, l_max_index, angle)
        if (np.max(l_conv) <= conv_threshold) & (angle < 10) & ((l_min_index == 0) | (l_max_index ==warped.shape[1])):
            break
        
        
        direction=new_center-l_center
        l_center = new_center
        
        # Add what we found for that layer
        window_centroids.append(l_center)
        new_y = int(warped.shape[0]-(level+0.5)*window_height)
        y_values.append(new_y if np.max(l_conv) > conv_threshold else 0)
        conv.append(l_conv)

    return np.array(window_centroids), np.array(y_values), conv

def find_window_centroids(warped, window_width, window_height, margin):
    window = np.ones(window_width) # Create our window template that we will use for convolutions
   
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(4*warped.shape[0]/5):,:int(warped.shape[1]/2)], axis=0)
    l_conv = np.convolve(window, l_sum)
    l_center = np.argmax(l_conv)-window_width/2
    r_sum = np.sum(warped[int(4*warped.shape[0]/5):,int(warped.shape[1]/2):], axis=0)
    r_conv = np.convolve(window, r_sum)
    r_center = np.argmax(r_conv)-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    new_y = warped.shape[0]-int(window_height/2)
    
    x_left, y_left, conv_left = find_line_centroids(warped, l_center, new_y, l_conv, window_width, window_height, margin)
    x_right, y_right, conv_right = find_line_centroids(warped, r_center, new_y, r_conv, window_width, window_height, margin)

    return np.array(x_left), np.array(y_left), conv_left, np.array(x_right), np.array(y_right), conv_right

def draw_result(warped, window_centroids, y_centroids,  window_width, window_height):
    zero_channel = np.zeros_like(warped)
    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        points = np.zeros_like(warped)

        # Go through each level and draw the windows
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            mask = window_mask(window_width,window_height,warped,window_centroids[level],y_centroids[level])
            # Add graphic points from window mask here to total pixels found 
            points[(points == 255) | ((mask == 1) ) ] = 255

        # Draw the results
        template = np.array(points, np.uint8)
        
        # make window pixels green
        template = np.array(cv2.merge((zero_channel, template, zero_channel)), np.uint8) 
    else: # If no window centers found, just display orginal road image
        template = np.array(cv2.merge((zero_channel, zero_channel, zero_channel)), np.uint8)
    return template

def fit_polynmial(subplot, warped, left_x, right_x, y_values_l, y_values_r):
    left_fit = np.polyfit(y_values_l, left_x, 2)
    right_fit = np.polyfit(y_values_r, right_x, 2)
    
    image_height = warped.shape[0]
    plot_y = np.linspace(0, image_height - 1, image_height)
    left_fit_x = left_fit[0]*plot_y**2+left_fit[1]*plot_y+left_fit[2]
    right_fit_x = right_fit[0]*plot_y**2+right_fit[1]*plot_y+right_fit[2]
    
    y_eval_l = np.max(y_values_l)
    y_eval_r = np.max(y_values_r)
    left_curverad = ((1+(2*left_fit[0]*y_eval_l+left_fit[1])**2)**1.5)/np.absolute(2*left_fit[0])
    right_curverad = ((1+(2*right_fit[0]*y_eval_r+right_fit[1])**2)**1.5)/np.absolute(2*right_fit[0])
    print(left_curverad, right_curverad)
    
    subplot.imshow(warped, cmap="gray")
    subplot.set_title('Centroids and Line', fontsize=20)
    subplot.plot(left_fit_x, plot_y, color='yellow', linewidth=1)
    subplot.plot(right_fit_x, plot_y, color='yellow', linewidth=1)
    subplot.scatter(left_x, y_values_l, c='red', s=20)
    subplot.scatter(right_x, y_values_r, c='green', s=20)
    subplot.set_xlim(0, 1280)
    subplot.set_ylim(7200, 0)
    return plot_y, left_fit_x, right_fit_x

def improve_lines(subplot, warped, left_x, right_x, y_values_l, y_values_r):
    left_fit = np.polyfit(y_values_l, left_x, 2)
    right_fit = np.polyfit(y_values_r, right_x, 2)
    
    x_left_f, y_left_f = ConvolutionalSlider.get_filtered_line(left_fit, left_x, np.array(y_values_l), 60)
    x_right_f, y_right_f = ConvolutionalSlider.get_filtered_line(right_fit, right_x, np.array(y_values_r), 60)
    
    left_fit = np.polyfit(y_left_f, x_left_f, 2)
    right_fit = np.polyfit(y_right_f, x_right_f, 2)
    
    image_height = warped.shape[0]
    
    plot_y = np.linspace(0, image_height - 1, image_height)
    left_fit_x = left_fit[0]*plot_y**2+left_fit[1]*plot_y+left_fit[2]
    right_fit_x = right_fit[0]*plot_y**2+right_fit[1]*plot_y+right_fit[2]
    
    subplot.imshow(warped, cmap='gray')
    subplot.set_title('Tuned Lines', fontsize=20)
    subplot.plot(left_fit_x, plot_y, color='yellow', linewidth=1)
    subplot.plot(right_fit_x, plot_y, color='yellow', linewidth=1)
    subplot.scatter(x_left_f, y_left_f, c='red', s=20)
    subplot.scatter(x_right_f, y_right_f, c='green', s=20)
    subplot.set_xlim(0, 1280)
    subplot.set_ylim(7200, 0)
    return plot_y, left_fit_x, right_fit_x
    
def draw_line_area(undistorted, warped, plot_y, left_plot_x, right_plot_x, x_left, x_right, y_values_l, y_values_r):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_plot_x, plot_y]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_plot_x, plot_y])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    
    left_line = np.stack((x_left.astype(np.int32), y_values_l), axis=-1)
    right_line = np.stack((x_right.astype(np.int32), y_values_r), axis=-1)

    #color_warp = cv2.polylines(color_warp, [left_line], False, (255, 0, 255), thickness=40)
    #color_warp = cv2.polylines(color_warp, [right_line], False, (255, 0, 0), thickness=40)

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    height = undistorted.shape[0]
    width = undistorted.shape[1]
    
    new_warp = cv2.warpPerspective(color_warp, Frame.inverse_matrix, (width, height))
    # Combine the result with the original image
    result = cv2.addWeighted(undistorted, 1, new_warp, 0.3, 0)
    
    plt.figure();
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(36, 9))
    f.tight_layout()
    ax1.imshow(undistorted)
    ax1.set_title('Original Image', fontsize=20)
    ax2.imshow(color_warp)
    ax2.set_title('Road', fontsize=20)
    ax3.imshow(result)
    ax3.set_title('Result Image', fontsize=20)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    return result

Display line back on original image

In [ ]:
def identify_lines(image_name):
    image_path = os.path.join(test_folder_path, image_name)
    img = mpimg.imread(image_path)
    
    frame = Frame(img)
    warped = frame.process_frame()
    plt.figure();
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 20))
    f.tight_layout()
    
    x_left, y_values_l, conv_l, x_right, y_values_r, conv_r = find_window_centroids(warped, window_width, window_height, margin)
    x_left = x_left[y_values_l.nonzero()]
    x_right = x_right[y_values_r.nonzero()]
    y_values_l = y_values_l[y_values_l.nonzero()]
    y_values_r = y_values_r[y_values_r.nonzero()]
    
    output_l = draw_result(warped, x_left, y_values_l, window_width, window_height)
    output_r = draw_result(warped, x_right, y_values_r, window_width, window_height)
    
    # making the original road pixels 3 color channels
    warpage = (np.dstack(( warped, warped, warped))*255).astype(np.uint8)
    output = cv2.addWeighted(warpage, 0.8, output_l+output_r, 0.5, 0.0) # overlay the orignal road image with window results
    # Display the final results
    ax1.imshow(output)
    ax1.set_title('Sliding Window', fontsize=20)
    
    output_image_path = os.path.join(output_folder_path, 'window_'+image_name)
    mpimg.imsave(output_image_path, output)
    
    fit_polynmial(ax2, warped, x_left, x_right, y_values_l, y_values_r)
    
    plot_y, left_plot_x, right_plot_x = improve_lines(ax3, warped, x_left, x_right, y_values_l, y_values_r)
    
    result = draw_line_area(frame.img, warped, plot_y, left_plot_x, right_plot_x, x_left, x_right, y_values_l, y_values_r)

    '''
    f,a = plt.subplots(len(conv_l),1)
    a = a.ravel()
    for idx,ax in enumerate(a):
        cur_conv = conv_l[idx]
        #print(cur_conv)
        x = range(0, len(cur_conv))
        ax.plot(x, cur_conv.astype(np.int32))
        ax.set_ylim([0, max(cur_conv)])

    f,a = plt.subplots(len(conv_r),1)
    a = a.ravel()
    for idx,ax in enumerate(a):
        cur_conv = conv_r[idx]
        #print(cur_conv)
        x = range(0, len(cur_conv))
        ax.plot(x, cur_conv.astype(np.int32))
        ax.set_ylim([0, max(cur_conv)])  
    '''
    
    output_image_path = os.path.join(output_folder_path, 'result_'+image_name)
    mpimg.imsave(output_image_path, result)

for name in images:
    identify_lines(os.path.basename(name))